# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
from PIL import Image 

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 15 #experiment with the batch size

In [4]:
print(train_doc.shape)
print(val_doc.shape)

(663,)
(100,)


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
#Image size to consider
final_img_size=(120,120)
img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]
total_img = len(img_idx)
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),final_img_size[0],final_img_size[1],3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if (image.shape[1]==160):
                        image = image[:,20:140,:]
                    else:
                        image = imresize(arr=image, size=final_img_size)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]-np.percentile(image[:,:,0],5))/(np.percentile(image[:,:,0],95)-np.percentile(image[:,:,0],5))
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]-np.percentile(image[:,:,1],5))/(np.percentile(image[:,:,1],95)-np.percentile(image[:,:,1],5))
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]-np.percentile(image[:,:,2],5))/(np.percentile(image[:,:,2],95)-np.percentile(image[:,:,2],5))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size!=0):
            batch_data = np.zeros((len(t)%batch_size,len(img_idx),final_img_size[0],final_img_size[1],3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch_size* num_batches)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch_size*num_batches)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = imresize(arr=image, size=final_img_size)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]-np.percentile(image[:,:,0],5))/(np.percentile(image[:,:,0],95)-np.percentile(image[:,:,0],5))
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]-np.percentile(image[:,:,1],5))/(np.percentile(image[:,:,1],95)-np.percentile(image[:,:,1],5))
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]-np.percentile(image[:,:,2],5))/(np.percentile(image[:,:,2],95)-np.percentile(image[:,:,2],5))
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


In [7]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

## Model - Conv3D
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,SpatialDropout3D,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
#write your model here
##########################
num_classes = 5
learning_rate = 0.001
validation_split = 0.2
verbosity = 1
##########################

Conv3D_Model = Sequential()
Conv3D_Model.add(Conv3D(16,(3,3,3), padding='same',activation='relu',input_shape=(total_img, 120, 120, 3)))
Conv3D_Model.add(BatchNormalization())
Conv3D_Model.add(MaxPooling3D(pool_size=(2,2,2),strides=(1, 2, 2)))


Conv3D_Model.add(Conv3D(32,(3,3,3),padding='same',activation='relu'))
Conv3D_Model.add(BatchNormalization())
Conv3D_Model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))

Conv3D_Model.add(Conv3D(64, (3,3,3),padding='same',activation='relu'))
Conv3D_Model.add(BatchNormalization())
Conv3D_Model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))

Conv3D_Model.add(Flatten())
Conv3D_Model.add(Dense(128,activation='relu'))
Conv3D_Model.add(Dropout(0.4))
Conv3D_Model.add(Dense(5, activation='softmax'))


Conv3D_Model.summary()
#optimizers,losses,leraning rates, epochs.,kernel initializers

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 120, 120, 16)  1312      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 14, 60, 60, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 14, 60, 60, 32)    13856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 60, 60, 32)    128       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 7, 30, 30, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 30, 30, 64)     55360     
__________

In [10]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
Conv3D_Model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
model_name = 'Conv3D_Model' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR =  ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.3,
                                   cooldown=0,
                                   patience=2,
                                   mode='min',
                                   verbose=1,
                                   min_lr=0.00001)     # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [13]:
Conv3D_Model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,callbacks=callbacks_list)

Source path =  ./Project_data/val ; batch size = 15
Source path =  ./Project_data/train ; batch size = 15
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 6/45 [===>..........................] - ETA: 1:25 - loss: 6.3729 - categorical_accuracy: 0.1222

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


45/45 [==============================] - 64s 1s/step - loss: 2.1870 - categorical_accuracy: 0.2949 - val_loss: 1.2580 - val_categorical_accuracy: 0.4100

Epoch 00001: val_loss improved from inf to 1.25797, saving model to Conv3D_Model_2020-05-0312_29_03.496105/model-00001-2.19359-0.30015-1.25797-0.41000.h5
Epoch 2/20
45/45 [==============================] - 56s 1s/step - loss: 1.2022 - categorical_accuracy: 0.4859 - val_loss: 1.1612 - val_categorical_accuracy: 0.5300

Epoch 00002: val_loss improved from 1.25797 to 1.16123, saving model to Conv3D_Model_2020-05-0312_29_03.496105/model-00002-1.20585-0.48265-1.16123-0.53000.h5
Epoch 3/20
45/45 [==============================] - 57s 1s/step - loss: 0.8933 - categorical_accuracy: 0.6460 - val_loss: 0.8728 - val_categorical_accuracy: 0.6300

Epoch 00003: val_loss improved from 1.16123 to 0.87283, saving model to Conv3D_Model_2020-05-0312_29_03.496105/model-00003-0.88461-0.65158-0.87283-0.63000.h5
Epoch 4/20
45/45 [============================

## Model - Conv2D + RNN
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [14]:
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 50


In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten,Activation,Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16

#write your model here
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
base_model_output = base_model.outputs
flattened_output = Flatten()(base_model_output)
conv_features = Dense(128,activation='relu')(flattened_output)

conv_model = Model(inputs=base_model.inputs,outputs=conv_features)

for layer in base_model.layers:
    layer.trainable = False

rnn_model = Sequential()
rnn_model.add(TimeDistributed(conv_model,input_shape=(15,120,120,3)))
rnn_model.add(GRU(64, return_sequences=True))
rnn_model.add(GRU(32, return_sequences=True))
rnn_model.add(Dropout(0.5))
rnn_model.add(GRU(32))
rnn_model.add(Dropout(0.5))
rnn_model.add(Dense(16, activation='relu'))
rnn_model.add(Dense(5, activation='softmax'))

In [16]:
sgd = optimizers.SGD(lr=0.01, momentum=0.7, decay=1e-6, nesterov=True)
rnn_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (rnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 128)           15304640  
_________________________________________________________________
gru_1 (GRU)                  (None, 15, 64)            37056     
_________________________________________________________________
gru_2 (GRU)                  (None, 15, 32)            9312      
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 32)            0         
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
__________

In [17]:
model_name = 'model_conv_rnn' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=2, verbose=1, mode='min',cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint,LR]

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [19]:
rnn_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


45/45 [==============================] - 72s 2s/step - loss: 1.6135 - categorical_accuracy: 0.2412 - val_loss: 1.6004 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60042, saving model to model_conv_rnn_2020-05-0312_29_03.496105/model-00001-1.61865-0.22775-1.60042-0.21000.h5
Epoch 2/50
45/45 [==============================] - 55s 1s/step - loss: 1.6387 - categorical_accuracy: 0.2044 - val_loss: 1.6014 - val_categorical_accuracy: 0.2400

Epoch 00002: val_loss did not improve from 1.60042
Epoch 3/50
45/45 [==============================] - 59s 1s/step - loss: 1.6240 - categorical_accuracy: 0.1882 - val_loss: 1.5963 - val_categorical_accuracy: 0.2300

Epoch 00003: val_loss improved from 1.60042 to 1.59634, saving model to model_conv_rnn_2020-05-0312_29_03.496105/model-00003-1.62109-0.19155-1.59634-0.23000.h5
Epoch 4/50
45/45 [==============================] - 60s 1s/step - loss: 1.5915 - categorical_accuracy: 0.2503 - val_loss: 1.6253 - val_categorical_ac


Epoch 00030: val_loss did not improve from 0.61386

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.0012959999497979878.
Epoch 31/50
45/45 [==============================] - 60s 1s/step - loss: 0.3225 - categorical_accuracy: 0.8964 - val_loss: 1.0628 - val_categorical_accuracy: 0.6000

Epoch 00031: val_loss did not improve from 0.61386
Epoch 32/50
45/45 [==============================] - 59s 1s/step - loss: 0.2843 - categorical_accuracy: 0.9127 - val_loss: 0.6539 - val_categorical_accuracy: 0.7400

Epoch 00032: val_loss did not improve from 0.61386

Epoch 00032: ReduceLROnPlateau reducing learning rate to 0.0007775999838486314.
Epoch 33/50
45/45 [==============================] - 61s 1s/step - loss: 0.2739 - categorical_accuracy: 0.9186 - val_loss: 0.7637 - val_categorical_accuracy: 0.7000

Epoch 00033: val_loss did not improve from 0.61386
Epoch 34/50
45/45 [==============================] - 58s 1s/step - loss: 0.2574 - categorical_accuracy: 0.9333 - val_loss: 0.6121 - val